# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the csv output file from WeatherPy
weather_df = pd.read_csv("WeatherPy.csv")

# Determine if there are any missing values
weather_df.count()

Unnamed: 0         580
City               580
Cloudiness         580
Country            575
Date               580
Humidity           580
Lat                580
Lng                580
Max Temp           580
Temperature (F)    580
Wind Speed         580
dtype: int64

In [3]:
# Drop missing values
weather_df.dropna(inplace=True)
weather_df.count()

Unnamed: 0         575
City               575
Cloudiness         575
Country            575
Date               575
Humidity           575
Lat                575
Lng                575
Max Temp           575
Temperature (F)    575
Wind Speed         575
dtype: int64

In [4]:
# Cleaned up dataframe
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Temperature (F),Wind Speed
0,0,bredasdorp,2.0,ZA,1586813742,67.0,-34.53,20.04,57.20,55.69,5.44
1,1,tiksi,0.0,RU,1586813742,87.0,71.69,128.87,-7.56,-7.56,6.22
2,2,yurginskoye,56.0,RU,1586813743,71.0,56.82,67.39,39.72,39.72,4.54
3,3,port elizabeth,60.0,ZA,1586813743,82.0,-33.92,25.57,59.00,59.00,9.17
4,4,barrow,90.0,US,1586813623,92.0,71.29,-156.79,26.60,26.60,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [48]:
# Setup latitude and longitude locations
all_locations = weather_df[["Lat", "Lng"]]

# Setup humidity as weight
humidity = weather_df["Humidity"].astype(float)

In [72]:
# Plot heatmap
fig = gmaps.figure(center=(25,0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(all_locations, weights=humidity, max_intensity=100, dissipating=False, point_radius=4)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow dataframe to ideal conditions
temp_df = weather_df.loc[(weather_df["Max Temp"] > 65) & (weather_df["Max Temp"] < 85) & (weather_df["Wind Speed"] < 10) & (weather_df["Humidity"] < 30),['City','Cloudiness','Country','Humidity','Lat','Lng','Max Temp','Wind Speed']]
temp_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed
8,gariaband,0.0,IN,20.0,20.64,82.06,79.95,2.26
224,baloda,0.0,IN,20.0,22.15,82.48,81.23,3.20
336,jaypur,0.0,IN,29.0,18.85,82.58,75.34,2.15
340,pachperwa,0.0,IN,25.0,27.52,82.65,78.96,7.52
344,sakakah,34.0,SA,18.0,29.97,40.21,68.00,6.93
372,quime,16.0,BO,16.0,-16.98,-67.22,67.44,5.91
430,bilma,0.0,NE,13.0,18.69,12.92,82.31,6.58
472,mopipi,26.0,BW,24.0,-21.18,24.88,79.54,5.64
484,gusau,2.0,NG,12.0,12.16,6.66,84.38,9.75
540,kaduna,63.0,NG,14.0,10.52,7.44,83.25,1.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
# Add Hotel Name column to hotel dataframe
hotel_df = temp_df.reset_index(drop=True)
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,gariaband,0.0,IN,20.0,20.64,82.06,79.95,2.26,
1,baloda,0.0,IN,20.0,22.15,82.48,81.23,3.20,
2,jaypur,0.0,IN,29.0,18.85,82.58,75.34,2.15,
3,pachperwa,0.0,IN,25.0,27.52,82.65,78.96,7.52,
4,sakakah,34.0,SA,18.0,29.97,40.21,68.00,6.93,
5,quime,16.0,BO,16.0,-16.98,-67.22,67.44,5.91,
6,bilma,0.0,NE,13.0,18.69,12.92,82.31,6.58,
7,mopipi,26.0,BW,24.0,-21.18,24.88,79.54,5.64,
8,gusau,2.0,NG,12.0,12.16,6.66,84.38,9.75,
9,kaduna,63.0,NG,14.0,10.52,7.44,83.25,1.14,


In [32]:
# Create the base url and parameters for the API query by city name
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

params = {
    'key': g_key,
    'type': 'lodging',
    'radius': 5000
}

# Use itterrows to go through the cities in the dataframe
for index, row in hotel_df.iterrows():
    
    # Get the city name from the dataframe
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    coordinates = f'{lat},{lng}'
    
    # Add city name to the params
    params['location'] = coordinates
    
    # Assemble to url and params and make the request
    response = requests.get(base_url,params=params)
    places_data = response.json()
    
    # Get the info I need and write it into the dataframe if it exists
    try:
        hotel_df.loc[index, 'Hotel Name'] = places_data["results"][0]["name"]
        print(f'Hotel name populated for {city}.')
    
    except (KeyError, IndexError):
        print(f'Unable to find location in API. Skipping...')

Hotel name populated for gariaband.
Hotel name populated for baloda.
Hotel name populated for jaypur.
Hotel name populated for pachperwa.
Hotel name populated for sakakah.
Hotel name populated for quime.
Hotel name populated for bilma.
Unable to find location in API. Skipping...
Hotel name populated for gusau.
Hotel name populated for kaduna.
Unable to find location in API. Skipping...


In [42]:
# Drop missing Hotel Names
hotel_df.replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)
hotel_df

,City,Cloudiness,Country,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,gariaband,0.0,IN,20.0,20.64,82.06,79.95,2.26,HOTEL CITY REGENCY
1,baloda,0.0,IN,20.0,22.15,82.48,81.23,3.20,Ganesh Anand Catering services Baloda
2,jaypur,0.0,IN,29.0,18.85,82.58,75.34,2.15,Hotel Hello Jeypore
3,pachperwa,0.0,IN,25.0,27.52,82.65,78.96,7.52,भगत जी
4,sakakah,34.0,SA,18.0,29.97,40.21,68.00,6.93,Fakhamat Al Orjoana Apartment
5,quime,16.0,BO,16.0,-16.98,-67.22,67.44,5.91,Hostal Colibri
6,bilma,0.0,NE,13.0,18.69,12.92,82.31,6.58,Expeditions Ténére voyages
8,gusau,2.0,NG,12.0,12.16,6.66,84.38,9.75,Crystal Merit Suites & Apartment
9,kaduna,63.0,NG,14.0,10.52,7.44,83.25,1.14,HAMDALA HOTEL


In [85]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [87]:
# Plot heatmap
fig = gmaps.figure(center=(25,0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(all_locations, weights=humidity, max_intensity=100, dissipating=False, point_radius=4)

# Add heat layer
fig.add_layer(heat_layer)

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))